In [47]:
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup 
from selenium import webdriver

from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


import requests

import re

In [21]:
driver=webdriver.Chrome(r"C://chromedriver_win32/chromedriver.exe")

# Q1: Write a python program which searches all the product under a particular product from www.amazon.in. The product name to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [22]:
driver.get('https://www.amazon.in/')

In [23]:
search_bar = driver.find_element_by_id("twotabsearchtextbox")
search_bar.clear() 
search_bar.send_keys("Books")
search_button = driver.find_element_by_xpath('//span[@id="nav-search-submit-text"]')
search_button.click()  

# Q2: In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [24]:
brand_name = []
name_of_the_product = []
rating = []
no_of_ratings = []
price = []
return_exchange = []
expected_delivery = []
availability = []
other_details = []
product_URL = []

In [ ]:
for page in range(0,3):  
        
        prd_link = driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")
        for i in prd_link:
            product_URL.append(i.get_attribute('href'))
            
        next_button = driver.find_element_by_xpath("//li[@class='a-last']/a")
        if next_button.text == 'Next→':
            driver.get(next_button.get_attribute('href'))
            print("Product urls of page has been scraped.")
        else:
            break
   
    
for i in product_URL:
        driver.get(i)
        
        #product title and brand name
        title = driver.find_element_by_xpath("//span[@id='productTitle']")
        name_of_the_product.append(title.text)
        
        #number of ratings
        try:
            rat_tag = driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']")
            no_of_ratings.append(rat_tag.text)
        except NoSuchElementException as e:
            no_of_ratings.append('-')
            
            
        #ratings star
        try:
            rat_tag = driver.find_element_by_xpath("//div[@id='averageCustomerReviews']")
            rat_tag.click() #this should be done as selenium rating option is not visible and selenium needs visible data to scrape
            rat_n = driver.find_element_by_xpath("//span[@class='a-size-medium a-color-base']")
            rating.append(rat_n.text)
        
        except NoSuchElementException as e:
            rating.append('-')

        
        #price
        try:
            prc_tag = driver.find_element_by_xpath("//td[@class='a-span12']/span")
            price.append(prc_tag.text)
        except NoSuchElementException as e:
            price.append('-')

        
        #exchange/return
        try:
            ex_rt = driver.find_element_by_xpath("//div[@id='RETURNS_POLICY']")
            return_exchange.append(ex_rt.text)
        except NoSuchElementException as e:
            return_exchange.append('-')

            
        #delivery
        try:
            del_tag = driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
            expected_delivery.append(del_tag.text)
        except NoSuchElementException as e:
            expected_delivery.append('-')


        #availability
        try:
            avl_tag = driver.find_element_by_xpath("//div[@id='availability']")
            availability.append(avl_tag.text)
        except NoSuchElementException as e:
            availability.append('-')


        #brand_name
        try:
            brand = driver.find_element_by_xpath("//a[@id='bylineInfo']")
            brand_name.append(brand.text)
        except NoSuchElementException as e:
            brand_name.append('-')
        
        
        #other details
        try:                                                                                    
            dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
            other_details.append('  ||  '.join(dtls.text.split('\n')))
        except NoSuchElementException:
            other_details.append('-')

In [26]:
amazon = pd.DataFrame({
    'brand':brand_name,
    'name':name_of_the_product,
    'rating':rating,
    'number of ratings':no_of_ratings,
    'price':price,
    'return/exchange':return_exchange,
    'delivery':expected_delivery,
    'availability':availability,
    'other details': other_details
})


amazon.to_csv('amazon.csv')

amazon

,brand,name,rating,number of ratings,price,return/exchange,delivery,availability,other details
0,-,The Pursuit of Happiness: A Book of Studies an...,4.3 out of 5,293 ratings,-,10 Days Replacement Only,"Wednesday, April 28",In stock.,-
1,-,The Psychology of Money,4.6 out of 5,"8,327 ratings",-,10 Days Replacement Only,"Wednesday, April 28",In stock.,-
2,-,World’s Greatest Books For Personal Growth & W...,4.5 out of 5,"5,607 ratings",-,10 Days Replacement Only,"Wednesday, April 28",In stock.,-
3,-,Mandala: Colouring Books for Adults with Tear ...,4.5 out of 5,"1,006 ratings",-,10 Days Replacement Only,"Thursday, April 29",In stock.,-
4,-,101 Panchatantra Stories for Children: Colourf...,4.5 out of 5,922 ratings,-,10 Days Replacement Only,"Wednesday, April 28",In stock.,-
5,-,Great Stories for Children,4.5 out of 5,"3,968 ratings",-,10 Days Replacement Only,"Wednesday, April 28",In stock.,-
6,-,Harry Potter and the Philosopher's Stone,4.7 out of 5,"26,111 ratings",-,10 Days Replacement Only,"Wednesday, April 28",In stock.,-
7,-,101 Panchatantra Stories,4.3 out of 5,"3,845 ratings",-,10 Days Replacement Only,"Wednesday, April 28",In stock.,-
8,-,The Alchemist,4.6 out of 5,"51,164 ratings",-,10 Days Replacement Only,"Wednesday, April 28",In stock.,-
9,-,Refreshing Mandala - Colouring Book for Adults...,4.6 out of 5,"3,268 ratings",-,10 Days Replacement Only,"Thursday, April 29",,-


# Q3: Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [27]:
driver.get('https://images.google.com/')

In [28]:
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')
search_bar.send_keys("fruits")
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button') 
search_button.click()

In [32]:
for _ in range(400):
    driver.execute_script("window.scrollBy(0,10000)")

In [33]:
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

In [34]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

179

In [36]:
print("Wait for Downloading to complete...")
for i in range(len(img_urls)):
    if i >= 100:
        break
    response= requests.get(img_urls[i])
    file = open("C:/Users/MD/Desktop/Fliprobo/Web scraping assignment 3/banana/img"+str(i)+".jpg", "wb")
    file.write(response.content)
print("Downloading completed...")

Wait for Downloading to complete...


In [37]:
# Similarly we can search for 'cars' and 'machine learning' and scrape images using same method.

#### Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [38]:
driver.get('https://www.flipkart.com/')

In [39]:
login_button = driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]') 
login_button.click()

In [40]:
search_bar = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')
search_bar.send_keys("pixel")
search_button = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')
search_button.click()

In [42]:
product_url = []
brand_name = []
model_name = []
color = []
ram = []
rom = []
processor = []
no_processor = []
battery = []
price = []
prim_cam = []
sec_cam = []
disp_size = []
disp_res = []

In [43]:
url = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
for i in url:
    product_url.append(i.get_attribute('href'))

In [48]:
for i in product_url:
        driver.get(i)
        #scraping brand names:
        try:
            brand_tag = driver.find_element_by_xpath("//span[@class='B_NuCI']")
            brand_name.append(brand_tag.text.split(' ')[0])
        except NoSuchElementException:
            print('-')
        
        
    #scraping smartphone name
        try:
            phone_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][1]/table/tbody/tr[3]/td[2]//ul/li")
            model_name.append(phone_tag.text)
        except NoSuchElementException as e:
            model_name.append('-')

        #color of smartphone
        try:
            col_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][1]/table/tbody/tr[4]/td[2]//ul/li")
            color.append(col_tag.text)
        except NoSuchElementException as e:
            color.append('-')
            
            
        #clicking on readmore button
        try:
            WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH,"//button[@class='_2KpZ6l _1FH0tX']")))
            read_more = driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']")
            read_more.click()
        except NoSuchElementException as e:
            print("can't move to next page")
            
            
        #Ram and Rom
        try:
            ram_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/div")
            if ram_tag.text != "Memory & Storage Features" : raise NoSuchElementException
            ram_tag1 = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/table/tbody/tr[2]/td[2]/ul/li")
            ram.append(ram_tag1.text)
        except NoSuchElementException as e:
            ram.append('-')

         
        try:
            ram_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/div")
            if ram_tag.text != "Memory & Storage Features" : raise NoSuchElementException
            rom_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/table/tbody/tr[1]/td[2]/ul/li")
            rom.append(rom_tag.text)
        except NoSuchElementException as e:
            rom.append('-')


        #primary camera
        try:
            prm_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/div")
            if prm_tag.text != 'Camera Features': raise NoSuchElementException
            prm_tag1 = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[2]/td[2]/ul/li")
            prim_cam.append(prm_tag1.text)
        except NoSuchElementException as e:
            prim_cam.append('-')


        #secondary camera
        try:
            cam_tag_check = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[5]/td[1]")
            cam_tag_check2 = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[6]/td[1]")
            if cam_tag_check.text =='Secondary Camera':
                sec_cam.append(driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[5]/td[2]/ul/li").text)
            elif cam_tag_check2.text == 'Secondary Camera':
                sec_cam.append(driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[6]/td[2]/ul/li").text)
            else:
                sec_cam.append('-')
        except NoSuchElementException:
            sec_cam.append('-')


        #display size and resolution
        try:
            disp_size_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/div")
            if disp_size_tag.text != 'Display Features' : raise NoSuchElementException
            disp_size_data = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/table/tbody/tr[1]/td[2]/ul/li")
            disp_size.append(disp_size_data.text)
        except NoSuchElementException:
            disp_size.append('-')


        #display size and resolution
        try:
            disp_res_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/div")
            if disp_res_tag.text != 'Display Features' : raise NoSuchElementException
            disp_res_data = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/table/tbody/tr[2]/td[2]/ul/li")
            disp_res.append(disp_res_data.text)
        except NoSuchElementException:
            disp_res.append('-')

        
        #processor
        try:
            proc_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/div")
            if proc_tag.text != "Os & Processor Features": raise NoSuchElementException
            proc_name = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody/tr[2]/td[2]/ul/li")
            processor.append(proc_name.text)
        except NoSuchElementException:
            processor.append('-')


        #number of processor cores
        try:
            core_chek = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/div")
            if core_chek.text != "Os & Processor Features": raise NoSuchElementException
            core_1 = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody/tr[2]/td[1]")
            core_2 = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody/tr[3]/td[1]")
            if core_1.text == "Processor Core":
                no_processor.append(driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody/tr[2]/td[2]/ul/li").text)
            elif core_2.text == "Processor Core":
                no_processor.append(driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody/tr[3]/td[2]/ul/li").text)
            else:
                no_processor.append('-')
        except NoSuchElementException:
            no_processor.append('-')



        #battery capacity
        try:
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
                if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                    bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                    if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                    bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
                elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                    bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                    if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                    bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
                else:
                    raise NoSuchElementException
            else :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
            battery.append(bat_cap.text)
        except NoSuchElementException:
            battery.append('-')
    
    
    
        #price of the product
        try:
            prc_tag = driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
            price.append(prc_tag.text)
        except NoSuchElementException:
            price.append('-')

In [55]:
flipkart = pd.DataFrame({
    'brand':brand_name,
    'model':model_name,
    'color':color,
    'ram':ram,
    'storage':rom,
    'processor':processor,
    'num_processor':no_processor,
    'batter_capacity':battery,
    'primar_camera': prim_cam,
    'secondar_camera':sec_cam,
    'display_size':disp_size,
    'display_resolution':disp_res,
    'price':price,
    'product_url': product_url
})

flipkart.to_csv('flipkart.csv')

flipkart[0:20]

,brand,model,color,ram,storage,processor,num_processor,batter_capacity,primar_camera,secondar_camera,display_size,display_resolution,price,product_url
0,Google,Pixel 4a,Just Black,6 GB,128 GB,Qualcomm Snapdragon 730G,Octa Core,3140 mAh,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,Google,Pixel 4a,Just Black,4 GB,64 GB,Qualcomm Snapdragon 670,Octa Core,3000 mAh,12.2MP Rear Camera,8MP Front Camera,14.22 cm (5.6 inch),2220 x 1080 pixels,"₹39,999",https://www.flipkart.com/google-pixel-3a-clear...
2,Google,Pixel 4a,Just Black,4 GB,64 GB,MediaTek Helio G80,Octa Core,5020 mAh,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,"₹9,999",https://www.flipkart.com/redmi-9-prime-matte-b...
3,Google,Pixel 3a,Clearly White,4 GB,64 GB,Qualcomm Snapdragon 662,Octa Core,6000 mAh,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,"₹10,499",https://www.flipkart.com/redmi-9-power-blazing...
4,Redmi,Redmi 9 Prime,Matte Black,4 GB,64 GB,Qualcomm Snapdragon 662,Octa Core,6000 mAh,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,"₹10,499",https://www.flipkart.com/redmi-9-power-electri...
5,REDMI,9 Power,Blazing Blue,4 GB,64 GB,MediaTek Helio G85,Octa Core,5020 mAh,48MP Rear Camera,-,16.59 cm (6.53 inch),2340 x 1080$$pixel,"₹10,999",https://www.flipkart.com/redmi-note-9-aqua-gre...
6,REDMI,9 Power,Electric Green,4 GB,64 GB,MediaTek Helio G85,Octa Core,5020 mAh,48MP Rear Camera,-,16.59 cm (6.53 inch),2340 x 1080$$pixel,"₹10,999",https://www.flipkart.com/redmi-note-9-arctic-w...
7,REDMI,Note 9,Aqua Green,4 GB,64 GB,Qualcomm Snapdragon 662,Octa Core,6000 mAh,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,"₹10,499",https://www.flipkart.com/redmi-9-power-mighty-...
8,REDMI,Note 9,Arctic White,6 GB,128 GB,Qualcomm® Snapdragon™ 662,Octa Core,6000 mAh,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,"₹12,999",https://www.flipkart.com/redmi-9-power-mighty-...
9,REDMI,9 Power,Mighty Black,4 GB,64 GB,MediaTek Helio G85,Octa Core,5020 mAh,48MP Rear Camera,-,16.59 cm (6.53 inch),2340 x 1080$$pixel,"₹10,999",https://www.flipkart.com/redmi-note-9-pebble-g...


# Q5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [12]:
driver.get("https://www.google.co.in/maps")

In [13]:
city = input('Enter City: ')                                    
search = driver.find_element_by_id("searchboxinput")                
search.send_keys(city)                                      

button = driver.find_element_by_id("searchbox-searchbutton")            
button.click() 

Enter City: Mumbai


In [15]:
try:
    url= driver.current_url
    lat_lng = re.findall(r'@(.*)data',url_string)
    
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        lat = lat_lng_list[0]
        lng = lat_lng_list[1]
        print("Latitude= {},Longitude= {}".format(lat, lng))

except:
        print("Error")

Latitude= 19.0820391,Longitude= 72.6009774


# Q6.Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [73]:
driver.get('https://trak.in/')

In [74]:
#funding deal
driver.find_element_by_xpath("//li[@id='menu-item-51510']").click()

In [82]:
trak_dict = {}
trak_dict['Date'] = []
trak_dict['Startup Name'] = []
trak_dict['Industry/Vertical'] = []
trak_dict['Sub-Vertical'] = []
trak_dict['Location'] = []
trak_dict['Investor'] = []
trak_dict['Investment Type'] = []
trak_dict['Amount in USD '] = []

In [83]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount in USD '].append(a.text)

In [84]:
trak = pd.DataFrame(trak_dict)
trak

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


# Q7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in

In [72]:
driver.get('https://www.digit.in/')

In [58]:
top_10=driver.find_element_by_xpath("//div[@class='menu']/ul/li[3]/a")
top_10.click()

In [61]:
game = driver.find_element_by_xpath("//div[@class='listing_container']/ul/li[9]/a")
game.click()

In [69]:
name = []
price = []
OS = []
display = []
processor = []
HDD = []
RAM = []
weight = []
dimension = []
GPU = []

In [70]:
#names
names=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in names:
    name.append(i.text)
    
#os
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)
    
#display
displays=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in displays:
    display.append(i.text)
    
#processor
processors=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in processors:
    processor.append(i.text)
processor

#memory
memories=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")
memories_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]") 
for i in range(len(memories)):
        if memories[i].text=='Memory':
            HDD.append(memories_spec[i].text.split('/')[0])
            RAM.append(memories_spec[i].text.split('/')[1])
        else:
            HDD.append('No details available')
            RAM.append('No details available')

#weight
weights=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
weight_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(weights)):
        if weights[i].text=='Weight':
            weight.append(weight_spec[i].text)
        
#dimension
dimension=[]
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
dims_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(dims)):
        if dims[i].text=='Dimension':
            dimension.append(dims_spec[i].text)

#graphical processor
GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
GPUs_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(GPUs)):
        if GPUs[i].text=='Graphics Processor':
            GPU.append(GPUs_spec[i].text)
        
full_specs=[]
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))
    
for i in full_specs:
    driver.get(i)
    try:
        prices=driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(prices.text)
    except NoSuchElementException:
        price.append("No details available")

In [71]:
digit=pd.DataFrame({"Name":name,
                "Price":price,
                "OS":OS,
                "Display":display,
                "HDD":HDD,
                 "RAM":RAM,
                "processor":processor,
                "weight":weight,
                "Dimension":dimension,
                "Graphical processor":GPU})
digit.to_csv('digit.csv')
digit

,Name,Price,OS,Display,HDD,RAM,processor,weight,Dimension,Graphical processor
0,MSI GT76 TITAN DT 9SG,"379,990",WINDOWS 10 PRO,"17.3"" (UHD 3840X2160)",1 TB HDD,64 GBGB DDR4,INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080
1,ALIENWARE 17 AREA-51M,"422,000",WINDOWS 10 PRO,"17.3"" (FHD (1920 X 1080))",1 TB PCIe SSD,32GB DDR4,INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,3.87 Kg,42 mm x 402.6 mm x 319.14 mm,NVIDIA GeForce RTX 2080
2,HP OMEN 15 2020,"117,790",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,16 GBGB DDR4,INTEL I7-10750H 10TH GEN | 1.6GHZ,5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti
3,ASUS ZEPHYRUS G14,"164,990",WINDOWS 10 HOME,"14"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060
4,LENOVO LEGION Y540,"79,990",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",1 TB SSD,8GB DDR4,9TH GENERATION CORE INTEL I7-9750H | 2.6 GHZ,2.3,365mm x 260mm x 25.9mm,NVIDIA® GeForce RTX™ 2060
5,ASUS ROG ZEPHYRUS G GA502,"79,990",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,16GB DDR4,AMD RYZEN 7 QUAD CORE 3750H | 2.3 GHZ,2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti
6,ASUS ROG ZEPHYRUS S GX531,"239,990",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512GB SSD,16 GBGB DDR4,9TH GEN INTEL CORE I7-8750H | 2.2 GHZ,2.1,360 (W) x 268 (D) x 15.35~16.15 (H) mm,NVIDIA® GeForce RTX™ 2070 (Max-Q)
7,MSI GT83VR 7RE TITAN SLI,"349,990",WINDOWS 10 HOME 64 BIT,"18.4"" (1920 X 1080)",1.5 TB SATA,64GB DDR4,INTEL CM238 CORE I7-7820HK+CM238 7TH GEN | 3.5GHZ,5.5,458 x 339 x 69,Dual GTX1070
8,ASUS ROG ZEPHYRUS DUO 15,"244,990",WINDOWS 10,"15.6"" (3840 X 1100)",512 GB SSD,4 GBGB DDR4,INTEL CORE I7 10TH GEN 10875H | NA,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q
9,DELL G3,"61,990",WINDOWS 10 HOME PLUS,15.6 MP | NA,1TB HDD,8GB DDR4,8TH GEN INTEL CORE I5-8300H | 2.3GHZ,2.53,22.7 X 380 X 258,NVidia GeForce GTX 1050
